<a href="https://colab.research.google.com/github/Jhansipothabattula/Machine_Learning/blob/main/Day44.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Hyperparameter Tuning with Bayesian Optimization


## **Introduction to Bayesian Optimization**

### **What is Bayesian Optimization?**

* Advanced method for hyperparameter tuning that balances
  **exploration** (searching new regions) and
  **exploitation** (refining promising regions)
* Uses a probabilistic model to guide the search for optimal hyperparameters

### **How It Works**

* **Surrogate Model**

  * Builds a probabilistic model (e.g., Gaussian Process) of the objective function based on prior evaluations
* **Acquisition Function**

  * Balances exploration and exploitation by choosing the next hyperparameters to evaluate based on predicted performance and uncertainty
* **Iterative Refinement**

  * Updates the surrogate model after each evaluation, refining the search

### **Why Use Bayesian Optimization?**

* Efficient for high-dimensional and expensive-to-evaluate functions

* Reduces the number of evaluations required to find near-optimal hyperparameters




## **Using Libraries for Bayesian Optimization**

### **Popular Libraries**

### **Hyperopt**

* Simplifies Bayesian Optimization for hyperparameter tuning
* Works with **fmin** to minimize objective functions over a parameter space

### **Optuna**

* Flexible and user-friendly library for hyperparameter optimization
* Supports dynamic search spaces and pruning of unpromising trials




## **Understanding Exploration vs. Exploitation**

### **Exploration**

* Focuses on sampling hyperparameters from unexplored regions
* Useful for identifying new areas of high potential

### **Exploitation**

* Focuses on refining the search around regions with known high performance
* Useful for fine-tuning near-optimal hyperparameters

### **Bayesian Optimization’s Advantage**

* Balances these approaches using the acquisition function to minimize unnecessary evaluations while improving results



**Objective**

- Apply Bayesian Optimization using optuna to tune an XGBoost model and compare the results with Grid Search and Random Search

In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 9.1 MB/s eta 0:00:00


In [4]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import optuna

# Load The dataset
data = load_breast_cancer()
X , y = data.data, data.target

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardization features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

# Train a baseline XGBoost model
baseline_model = XGBClassifier(eval_metric="logloss", random_state=42)
baseline_model.fit(X_train, y_train)

# Evaluate the model
baseline_preds = baseline_model.predict(X_test)
baseline_accuracy = accuracy_score(y_test, baseline_preds)

# Define the objective function for optuna
def objective(trial):
  params = {
      "n_estimators": trial.suggest_int("n_estimators", 50, 500),
      "max_depth": trial.suggest_int("max_depth", 3, 100),
      "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
      "subsample": trial.suggest_float("subsample", 0.01, 0.3),
      "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
      "gamma": trial.suggest_float("gamma", 0, 5),
      "reg_alpha": trial.suggest_float("reg_alpha", 0, 10),
      "reg_lambda": trial.suggest_float("reg_lambda", 0, 10),
  }

  # Train XGBoost model with suggested params
  model = XGBClassifier(eval_metric="logloss", random_state=42, **params)
  model.fit(X_train, y_train)

  # Evaluate model on validation set
  preds = model.predict(X_test)
  accuracy = accuracy_score(y_test, preds)
  return accuracy

# Create an optuna study
study = optuna.create_study(direction="maximize")

# Optimize the objective function
study.optimize(objective, n_trials=50)

# Best parameters
print("Best Parameters:", study.best_params)

# Best accuracy
print("Best Accuracy:", study.best_value)

# Define parameter grid for Grid Search
param_grid_gs={
    "n_estimators": [100,200, 300],
    "max_depth":[3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.2],
    "subsample": [0.5, 0.8, 1.0]
}

# Train XGBoost with Grid Search
grid_search = GridSearchCV(
    estimator=XGBClassifier(eval_metric="logloss", random_state=42),
    param_grid=param_grid_gs,
    scoring="accuracy",
    cv=3,
    verbose=1,
)
grid_search.fit(X_train, y_train)

# Best parameters and accuracy
print("Best Parameters (Grid Search):", grid_search.best_params_)
print("Best Accuracy (Grid Search):", grid_search.best_score_)

# Define parameter distribution for Random Search
param_grid_rs={
    "n_estimators": [50, 100, 200, 300, 400],
    "max_depth": [3, 5, 7, 9],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "subsample": [0.6, 0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.6, 0.7, 0.8, 0.9, 1.0]
}

# Train XGBoost with Random Search
random_search = RandomizedSearchCV(
    estimator=XGBClassifier(eval_metric="logloss", random_state=42),
    param_distributions=param_grid_rs,
    n_iter=50,
    scoring="accuracy",
    cv=3,
    verbose=1,
    random_state=42
)
random_search.fit(X_train, y_train)

# Best parameters and accuracy
print("Best Parameters (Random Search):", random_search.best_params_)
print("Best Accuracy (Random Search):", random_search.best_score_)

[I 2025-12-11 03:09:15,158] A new study created in memory with name: no-name-98871d61-0684-45e4-a73a-e0f4c6add833


Training data shape: (455, 30)
Test data shape: (114, 30)


[I 2025-12-11 03:09:15,241] Trial 0 finished with value: 0.9385964912280702 and parameters: {'n_estimators': 175, 'max_depth': 32, 'learning_rate': 0.08927839906948845, 'subsample': 0.1530653745219995, 'colsample_bytree': 0.9175858547959228, 'gamma': 3.5199093864518325, 'reg_alpha': 5.920223680150318, 'reg_lambda': 4.848822758463214}. Best is trial 0 with value: 0.9385964912280702.
[I 2025-12-11 03:09:15,374] Trial 1 finished with value: 0.9385964912280702 and parameters: {'n_estimators': 276, 'max_depth': 80, 'learning_rate': 0.01864532878383155, 'subsample': 0.20703421950000925, 'colsample_bytree': 0.8492389250592778, 'gamma': 2.9179405768717652, 'reg_alpha': 7.883568171554765, 'reg_lambda': 8.11832225429603}. Best is trial 0 with value: 0.9385964912280702.
[I 2025-12-11 03:09:15,453] Trial 2 finished with value: 0.956140350877193 and parameters: {'n_estimators': 259, 'max_depth': 64, 'learning_rate': 0.2796429532934642, 'subsample': 0.06624824424065076, 'colsample_bytree': 0.7281390

Best Parameters: {'n_estimators': 488, 'max_depth': 19, 'learning_rate': 0.14821530817482642, 'subsample': 0.18314877065461976, 'colsample_bytree': 0.9864018870226585, 'gamma': 0.42660861492748003, 'reg_alpha': 2.6906670080936235, 'reg_lambda': 4.113214608937609}
Best Accuracy: 0.9824561403508771
Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best Parameters (Grid Search): {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.5}
Best Accuracy (Grid Search): 0.9735825490879516
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best Parameters (Random Search): {'subsample': 0.7, 'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.2, 'colsample_bytree': 1.0}
Best Accuracy (Random Search): 0.975775531544092
